Importar librerias


In [ ]:
import pandas as pd #libreria de python para manipular datos
import numpy as np # libreria de python para calculos numericos y analisis
import os #modulo para usar funcionalidades del sistema operativo
from pathlib import Path # modulo de la ruta que se usa
from heapq import merge # modulo para unir Dataframes
import re
import datetime
from datetime import datetime
pd.set_option('display.max_columns',None)#Cuantas columnas mostar por Dataframe

In [ ]:
separador = os.path.sep #busca el separador que sería / o \
current_dir = os.path.dirname(os.path.realpath('CruceUsuariosAPPrezagos.ipynb'))  #busca la ruta del archivo en el que está CruceUsuariosAPP.ipynb
dir = separador.join(current_dir.split(separador)[:-1]) #se devuelve una carpeta
filenameT = os.path.join(dir, "Data\\SO")
filename = os.path.join(dir, "Resultado\\Primario") 
filenameS= os.path.join(dir,"Data\\USUARIOS")

Importar usuarios

In [ ]:
file_exists_Usuarios = os.path.exists(filename+"\\IdentidadesEncontradasconSSFF.csv")
print(file_exists_Usuarios) #muestra false o true 

if file_exists_Usuarios == True: #Si el archivo existe

    Usuarios=pd.read_csv(filename+"\\IdentidadesEncontradasconSSFF.csv", sep=';', encoding = "utf-8-sig" , engine='python') #importa el archivo csv separado por punto y comas
   
    Usuarios= Usuarios[['DocumentoRRHH', 'Identidad','NombreRRHH', 'CASO', 'FECHA DE RETIRO','TIPO DE CONTRATO' ,'Usuario_Aplicativo', 'Aplica Inactivacion']]
    UsuariosD = Usuarios[['DocumentoRRHH']]
    UsuariosD.drop(UsuariosD[(UsuariosD['DocumentoRRHH'] =='0') | (UsuariosD['DocumentoRRHH'] == 0)].index, inplace=True)
    Usuarios['DocumentoRRHH'] = Usuarios['DocumentoRRHH'].astype(str)
    Usuarios = Usuarios[Usuarios['Aplica Inactivacion']=="si"]
    Usuarios = Usuarios.drop('Aplica Inactivacion',  axis=1)
    Usuarios = Usuarios.drop('Usuario_Aplicativo',  axis=1)
    UsuariosD['DocumentoRRHH'] = UsuariosD['DocumentoRRHH'].astype(str)
    Usuarios["Identidad"]= Usuarios["Identidad"].str.lower()
    UsuariosD=UsuariosD.drop_duplicates() #Quita filas que tengan el mismo documento
    Identidades=Usuarios[['Identidad']]
    Identidades=Identidades.drop_duplicates()
    Identidades=Identidades['Identidad'].tolist()
    Documento=UsuariosD[['DocumentoRRHH']]
    # doc = Documento["DocumentoRRHH"].str.split(".", n=1, expand=True)
    # doc= pd.DataFrame(doc)
    # doc.columns = ['doc', 'punto']
    # Documento = pd.concat([Documento, doc], axis=1)
    # Documento=Documento[['doc']] #USAR SOLO ESAS TRES COLUMNAS
    # Documento.rename(columns={'doc':'DocumentoRRHH'}, inplace=True)
    Documento=Documento.drop_duplicates()
    Documento=Documento['DocumentoRRHH'].tolist()
    Usuarios1= Usuarios[['DocumentoRRHH', 'NombreRRHH', 'CASO', 'FECHA DE RETIRO','TIPO DE CONTRATO']]
    Usuarios2= Usuarios[['DocumentoRRHH', 'Identidad','NombreRRHH', 'CASO', 'FECHA DE RETIRO','TIPO DE CONTRATO']]
    
    

In [ ]:
filtro1 = pd.DataFrame()
file_exists_SO = os.path.exists(filenameT+"\\SO.xlsx")

print(file_exists_SO)

if file_exists_SO == True:
    so=pd.read_excel(filenameT+"\\SO.xlsx")
    so=so[['Column2', 'Column3', 'Column4', 'Column7',  'Column5']]
    so = so.fillna({'Column7': 'Activo'})
    so.rename(columns={'Column2': 'Nombre_SO', 'Column3':'Identidad', 'Column4':'Nombre', 'Column7':'Estado', 'Column5':'Documento'}, inplace=True)
    so['Identidad'] = so['Identidad'].astype(str)
    so['Documento'] = so['Documento'].astype(str)
    so["Identidad"] = so["Identidad"].str.lower()
    so['Documento'] = so['Documento'].astype(str).str.replace(".", "", regex=True)
    so['Nombre'] = so['Nombre'].astype(str).str.replace(".", "", regex=True)
    so["Identidad"] = [ele.lstrip(' ') for ele in so["Identidad"]]
    so["Identidad"] = [ele.rstrip(' ') for ele in so["Identidad"]]
    so["Identidad"] = [ele.lstrip('.') for ele in so["Identidad"]]
    so["Identidad"] = [ele.rstrip('.') for ele in so["Identidad"]]
    so["Documento"] = [ele.lstrip(' ') for ele in so["Documento"]]
    so["Documento"] = [ele.rstrip(' ') for ele in so["Documento"]]
    so["Documento"] = [ele.lstrip('.') for ele in so["Documento"]]
    so["Documento"] = [ele.rstrip('.') for ele in so["Documento"]]
    so['Estado'] = so['Estado'].astype(str).str.replace("Bloqueado", "Inactivo", regex=True)   
 

In [ ]:

filtroId = so[so.Identidad.isin(Identidades)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
filtroId=filtroId.assign(Cruce='Identidad') 
filtroId = filtroId.merge(Usuarios2, how='left', left_on='Identidad', right_on='Identidad')

total = filtroId
total= total.drop_duplicates(total.columns[~total.columns.isin(['Cruce'])], keep='first')
    
total.index = range(total.shape[0])

In [ ]:
file_exists_SO2 = os.path.exists(filenameT+"\\UserLocales_ServersMovil.csv")

print(file_exists_SO2)

if file_exists_SO2 == True:
    so2=pd.read_csv(filenameT+"\\UserLocales_ServersMovil.csv", sep=',', encoding = 'utf-8-sig' , engine='python' )
    so2=so2[['Server', 'UserName', 'Active']]
    so2.rename(columns={'Server': 'Nombre_SO', 'UserName':'Identidad', 'Active':'Estado'}, inplace=True)
    so2["Identidad"] = so2["Identidad"].str.lower()
    so2["Nombre_SO"] = so2["Nombre_SO"].str.lower()
    so2['Estado'] = so2['Estado'].astype(str).str.replace("False", "Inactivo", regex=True)
    so2['Estado'] = so2['Estado'].astype(str).str.replace("True", "Activo", regex=True)
    so2["Identidad"] = [ele.lstrip(' ') for ele in so2["Identidad"]]
    so2["Identidad"] = [ele.rstrip(' ') for ele in so2["Identidad"]]
    so2["Identidad"] = [ele.lstrip('.') for ele in so2["Identidad"]]
    so2["Identidad"] = [ele.rstrip('.') for ele in so2["Identidad"]]
    filtroId = so2[so2.Identidad.isin(Identidades)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtroId=filtroId.assign(Cruce='Identidad') 
    filtroId = filtroId.merge(Usuarios2, how='left', left_on='Identidad', right_on='Identidad')
    filtroId= filtroId.drop_duplicates(filtroId.columns[~filtroId.columns.isin(['Cruce'])], keep='first')
    total = pd.concat([total, filtroId])
    
    total.index = range(total.shape[0])

In [ ]:
file_exists_SO3 = os.path.exists(filenameT+"\\UserLocales_ServersFija.csv")

print(file_exists_SO3)

if file_exists_SO3 == True:
    so3=pd.read_csv(filenameT+"\\UserLocales_ServersFija.csv", sep=',', encoding = 'utf-8-sig' , engine='python' )
    so3=so3[['Server', 'UserName', 'Active']]
    so3.rename(columns={'Server': 'Nombre_SO', 'UserName':'Identidad', 'Active':'Estado'}, inplace=True)
    so3["Identidad"] = so3["Identidad"].str.lower()
    so3["Nombre_SO"] = so3["Nombre_SO"].str.lower()
    so3['Estado'] = so3['Estado'].astype(str).str.replace("False", "Inactivo", regex=True)
    so3['Estado'] = so3['Estado'].astype(str).str.replace("True", "Activo", regex=True)
    so3["Identidad"] = [ele.lstrip(' ') for ele in so3["Identidad"]]
    so3["Identidad"] = [ele.rstrip(' ') for ele in so3["Identidad"]]
    so3["Identidad"] = [ele.lstrip('.') for ele in so3["Identidad"]]
    so3["Identidad"] = [ele.rstrip('.') for ele in so3["Identidad"]]
    filtroId = so3[so3.Identidad.isin(Identidades)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtroId=filtroId.assign(Cruce='Identidad') 
    filtroId = filtroId.merge(Usuarios2, how='left', left_on='Identidad', right_on='Identidad')
    filtroId= filtroId.drop_duplicates(filtroId.columns[~filtroId.columns.isin(['Cruce'])], keep='first')
    total = pd.concat([total, filtroId])
    
    total.index = range(total.shape[0])

In [ ]:
file_exists_SO4 = os.path.exists(filenameT+"\\UserLocales_ServersClaroCO.csv")

print(file_exists_SO4)

if file_exists_SO4 == True:
    so4=pd.read_csv(filenameT+"\\UserLocales_ServersClaroCO.csv", sep=',', encoding = 'utf-8-sig' , engine='python' )
    so4=so4[['Server', 'UserName', 'Active']]
    so4.rename(columns={'Server': 'Nombre_SO', 'UserName':'Identidad', 'Active':'Estado'}, inplace=True)
    so4["Identidad"] = so4["Identidad"].str.lower()
    so4["Nombre_SO"] = so4["Nombre_SO"].str.lower()
    so4['Estado'] = so4['Estado'].astype(str).str.replace("False", "Inactivo", regex=True)
    so4['Estado'] = so4['Estado'].astype(str).str.replace("True", "Activo", regex=True)
    so4["Identidad"] = [ele.lstrip(' ') for ele in so4["Identidad"]]
    so4["Identidad"] = [ele.rstrip(' ') for ele in so4["Identidad"]]
    so4["Identidad"] = [ele.lstrip('.') for ele in so4["Identidad"]]
    so4["Identidad"] = [ele.rstrip('.') for ele in so4["Identidad"]]

    filtroId = so4[so4.Identidad.isin(Identidades)] #filtra de ClaroCo todos los usuarios que existen en la lista (cedulas)
    filtroId=filtroId.assign(Cruce='Identidad') 
    filtroId = filtroId.merge(Usuarios2, how='left', left_on='Identidad', right_on='Identidad')
    filtroId= filtroId.drop_duplicates(filtroId.columns[~filtroId.columns.isin(['Cruce'])], keep='first')
    total = pd.concat([total, filtroId])
    
    total.index = range(total.shape[0])

In [ ]:
total

In [ ]:
def asignarSistema (servidor):
    servidores = {'amxprdecc': 'SAP 6.0', 'AMXPRDECC':'SAP 6.0','lnxremedy':'MyIT', 'colbtasga':'SGA', 'com103':'MAGUARE', 'comhp01':'SICACOM',
    'comhp06':'SICACOM','comhp15':'SICACOM',
    'comhp3':'AC - BSCS - MOTOR DE PAGOS', 'comhp43':'MOTOR DE PAGOS', 'comibm10':'MAHINDRA', 'comibm17':'CALLIDUS', 'comibm18':'CALLIDUS',
    'lnxpretup':'MAHINDRA', 'lnxracdbpr':'CA', 'comsun10':'MAGUARE', 'lnxrmmitprd':'MyIT','comsun7':'INTEGRADOR WEB', 'comsun':'CALLIDUS',
    'el01cn':'OIM', 'GSRCMDBTRI3':'MAXIMO', 'lnxdbussd':'MAHINDRA', 'lnxdb':'SGA', 'lnxgwussd':'MAHINDRA', 'lnxm':'INTEC','LNXM':'INTEC',
    'lnxo':'INTEGRADOR WEB', 'lnxrrsso':'MyIT', 'lnxsoa12':'INTEGRADOR WEB', 'LNXFTP02':'INTEGRADOR WEB', 'wpol':'POLIEDRO', 'wcdbrmm':'MyIT',
    'wcosql':'CRM-ONIX', 'colbtafile':'SGA'}
    for c in servidores:
        s = servidor.find(c)
        if s != -1:
            return servidores[c]

if not total.empty : 
    total['Nombre_SO']=total['Nombre_SO'].astype(str)
    total['Nombre_Sistema'] = total.apply(lambda x: asignarSistema(x['Nombre_SO']), axis=1)
    total= total[['Nombre_SO', 'Nombre_Sistema', 'Identidad', 'Nombre', 'Estado', 'Documento', 'Cruce', 'DocumentoRRHH', 'NombreRRHH', 'CASO', 'FECHA DE RETIRO', 'TIPO DE CONTRATO']]

In [ ]:
file_exists_SSFF_Activos = os.path.exists(filenameS+"\\Informe_Activos_Claro.xlsx")#Pregunta si en la ruta existe el archivo

print(file_exists_SSFF_Activos)#muestra false o true 

if file_exists_SSFF_Activos == True:#Si el archivo existe

    SsffA=pd.read_excel(filenameS+"\\Informe_Activos_Claro.xlsx")
    SsffA.columns = SsffA.columns.str.strip()
    SsffA=SsffA[['Cédula','Detalles de empleo Fecha de ingreso','Estado de colaborador']] #Usar solo las columnas que necesitamos
    SsffA.rename(columns={'Estado de colaborador':'STATUS SSFF', 'Cédula':'DocumentoRRHH', 'Detalles de empleo Fecha de ingreso':'Fecha Ingreso SSFF'}, inplace=True)
    SsffA.insert(loc = 3,column = 'Fecha Terminacion SSFF',value = '0')
    SsffA=SsffA[['STATUS SSFF',  'DocumentoRRHH', 'Fecha Ingreso SSFF', 'Fecha Terminacion SSFF']]

    SsffA = SsffA.sort_values('Fecha Ingreso SSFF', ascending=False)
    SsffA['Fecha Ingreso SSFF'] = pd.to_datetime(SsffA['Fecha Ingreso SSFF'], format="%d/%m/%Y %H:%M:%S" , errors='coerce')
    SsffA['Fecha Ingreso SSFF'] = SsffA['Fecha Ingreso SSFF'].dt.floor('D')
    pd.options.display.float_format = '{:.0f}'.format
    SsffA['DocumentoRRHH']= SsffA['DocumentoRRHH'].astype('Int64')

    SsffA['DocumentoRRHH'] = SsffA['DocumentoRRHH'].astype(str)

    SsffA = SsffA.dropna(subset=["DocumentoRRHH"])
    
    SsffA['DocumentoRRHH'] = [ele.lstrip('0') for ele in SsffA['DocumentoRRHH']]
    SsffA['DocumentoRRHH'] = [ele.lstrip(' ') for ele in SsffA['DocumentoRRHH']]
    SsffA['DocumentoRRHH'] = [ele.rstrip(' ') for ele in SsffA['DocumentoRRHH']]

    SsffA=SsffA.drop_duplicates(SsffA.columns[~SsffA.columns.isin(['Fecha Ingreso SSFF'])], keep='first')
    SsffA=SsffA[['STATUS SSFF',  'DocumentoRRHH', 'Fecha Ingreso SSFF', 'Fecha Terminacion SSFF']]

    total = total.merge(SsffA, how='left', left_on='DocumentoRRHH', right_on='DocumentoRRHH')
    total = total.fillna({'STATUS SSFF': 'Inactivo','FECHA DE RETIRO': 'N/A', 'Fecha Ingreso SSFF':'0', 'Fecha Terminacion SSFF':'0'}) 
    
    # total['Fecha Terminacion SSFF'] = pd.to_datetime(Nube['Fecha Terminacion SSFF'], format="%d/%m/%Y" , errors='ignore')
    # Nube['FECHA DE RETIRO'] = pd.to_datetime(Nube['FECHA DE RETIRO'], format="%Y/%m/%d" , errors='coerce')
    total=total.drop_duplicates(total.columns[~total.columns.isin(['STATUS SSFF', 'Fecha Ingreso SSFF', 'Fecha Terminacion SSFF'])], keep='first')
 

In [ ]:
total=total.assign(Reintegro='N/A')
total['Nombre']=total['Nombre'].str.lower()

In [ ]:
total

In [ ]:
if not total.empty== True:
    total.insert(loc=13, column='Aplica inactivacion', value  = 'si')
    # total.loc[(total['coincidencia nombre APP vs SSFF'] >= 25) , 'Aplica inactivacion']='si'

In [ ]:
if not total.empty== True: 
    total= total[['Nombre_SO','Nombre_Sistema', 'Identidad', 'Nombre', 'Estado', 'Documento', 'Cruce',  'DocumentoRRHH','NombreRRHH', 'CASO',
                'FECHA DE RETIRO', 'TIPO DE CONTRATO', 'STATUS SSFF', 'Fecha Ingreso SSFF',
                'Fecha Terminacion SSFF', 'Reintegro', 'Aplica inactivacion']]

In [ ]:
total

In [ ]:
fecha_actual = str(datetime.now().date())
hora_actual = datetime.now().time()
hora_actual = hora_actual.strftime('%H-%M-%S')

nombref=filename+'\\IdentidadesSOconSSFF '+fecha_actual+' '+hora_actual+'.csv'
total.to_csv(nombref,index=False,sep=';',encoding='utf-8-sig')
nombre1=filename+'\\IdentidadesSOconSSFF.csv'
total.to_csv(nombre1,index=False,sep=';',encoding='utf-8-sig')

In [ ]:
total